In [1]:
from itertools import groupby
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import isomorphism
import matplotlib.pyplot as plt
import re

edges = pd.DataFrame(columns=['Top','Right','Bottom','Left'])
full_tiles = {}

with open('day20input.txt') as f_data:    
    for k, g in groupby(f_data, lambda x: x.startswith('T')):
        if not k:
            tile = np.array([[str(x) for x in d.split()] for d in g if len(d.strip())])
            full_tiles[tile_num] = np.array([np.array([i for i in j[0]]) for j in tile])
            edges.loc[tile_num,'Top'] = tile[0][0]
            edges.loc[tile_num,'Bottom'] = tile[-1][0]
            edges.loc[tile_num,'Left'] = ''.join([i[0][0] for i in tile])
            edges.loc[tile_num,'Right'] = ''.join([i[0][-1] for i in tile])
        else:
            tile_num = [i for i in g][0][-6:-2]
            edges.loc[tile_num] = 0

for i in edges.columns:
    edges[i+'_f'] = edges[i].str[::-1]

In [2]:
#Part 1: What is the value of the product of the four corners' tile numbers?

counts = edges.stack().value_counts()
Unique = pd.Series(index=edges.index,data=0)

for i in edges.index:
    for j in edges.columns:
        if counts[edges.loc[i,j]] == 1:
            Unique[i] += 1
            
print(np.prod(Unique[Unique == 4].index.astype(int)))

18411576553343


In [3]:
#Part 2: How many #s are in the water apart from the sea monsters?

G = nx.Graph()
node_list = list(edges.index)
edge_list = []
for i in edges.stack().unique():
    if len(np.where(edges == i)[0]) == 2:
        edge_list.append((edges.index[np.where(edges == i)[0][0]],edges.index[np.where(edges == i)[0][1]]))
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

Layout = np.zeros((12,12))

H = nx.grid_2d_graph(12,12)
GM = isomorphism.GraphMatcher(G,H)
GM.is_isomorphic()
for i in GM.mapping.keys():
    Layout[GM.mapping[i][0],GM.mapping[i][1]] = i
    
Layout = Layout.astype(int).astype(str)

Corner1 = Layout[0,0]
next_piece = Layout[0,1]
next_edges = list(edges.loc[next_piece])
while counts[''.join(full_tiles[Corner1][0])] + counts[''.join(full_tiles[Corner1][:,0])] > 2:
    full_tiles[Corner1] = np.rot90(full_tiles[Corner1])
if ''.join(full_tiles[Corner1][:,-1]) not in next_edges:
    full_tiles[Corner1] = np.rot90(np.flip(full_tiles[Corner1],axis = 1))
    
for i in np.arange(1,12):
    L_connect = ''.join([j[-1] for j in full_tiles[Layout[0,i-1]]])
    
    for j in np.arange(3):
        if L_connect != ''.join([j[0] for j in full_tiles[Layout[0,i]]]):
            full_tiles[Layout[0,i]] = np.rot90(full_tiles[Layout[0,i]])

    if L_connect != ''.join([j[0] for j in full_tiles[Layout[0,i]]]):
        full_tiles[Layout[0,i]] = np.flip(full_tiles[Layout[0,i]],axis = 1)

    for j in np.arange(3):
        if L_connect != ''.join([j[0] for j in full_tiles[Layout[0,i]]]):
            full_tiles[Layout[0,i]] = np.rot90(full_tiles[Layout[0,i]])
                
for i in np.arange(1,12):
    for j in np.arange(12):
        U_connect = ''.join(full_tiles[Layout[i-1,j]][-1])
        
        for k in np.arange(3):
            if U_connect != ''.join(full_tiles[Layout[i,j]][0]):
                full_tiles[Layout[i,j]] = np.rot90(full_tiles[Layout[i,j]])

        if U_connect != ''.join(full_tiles[Layout[i,j]][0]):
            full_tiles[Layout[i,j]] = np.flip(full_tiles[Layout[i,j]],axis = 1)

        for k in np.arange(3):
            if U_connect != ''.join(full_tiles[Layout[i,j]][0]):
                full_tiles[Layout[i,j]] = np.rot90(full_tiles[Layout[i,j]])
                
full_puzzle = np.zeros((96,96)).astype(str)

for i in np.arange(12):
    for j in np.arange(12):
        full_puzzle[8*i:8*i+8,8*j:8*j+8] = full_tiles[Layout[i,j]][1:-1,1:-1]
        
full_puzzle[full_puzzle == '.'] = 0
full_puzzle[full_puzzle == '#'] = 1
monsters = []

for i in np.arange(4):
    total1 = 0
    total2 = 0
    for i in np.arange(96):
        total1 += len(re.findall('(1(0|1){4}1){3}11', ''.join(full_puzzle[i])))
        total2 += len(re.findall('(1(0|1){2}){5}1', ''.join(full_puzzle[i])))
    full_puzzle = np.rot90(full_puzzle)
    monsters.append(min(total1,total2))
    
full_puzzle = np.flip(full_puzzle,axis=1)
for i in np.arange(4):
    total1 = 0
    total2 = 0
    for i in np.arange(96):
        total1 += len(re.findall('(1(0|1){4}1){3}11', ''.join(full_puzzle[i])))
        total2 += len(re.findall('(1(0|1){2}){5}1', ''.join(full_puzzle[i])))
    full_puzzle = np.rot90(full_puzzle)
    monsters.append(min(total1,total2))
    
print(full_puzzle.astype(int).sum() - max(monsters)*15)

2002
